In [31]:
import imaplib
import email
from email import policy
import os
import pandas as pd

# Credentials and server settings
username = ''
password = ''
imap_host = 'outlook.office365.com'
imap_port = 993

# Current working directory, assumes that ill be saving the csvs where i am running this code
save_directory = os.getcwd()

# File mapping based on subject line prefix. Change/add to fit specific use cases
file_mapping = {
    '1': '1.csv',
    '2': '2.csv',
    '3': '3.csv'
}

# Read the security key from key.txt
with open(os.path.join(save_directory, 'key.txt'), 'r') as key_file:
    key_phrase = key_file.read().strip()

# Ensure all target files exist or create empty ones with headers ['Column1', 'Column2']

headers = ['Column1', 'Column2']
for filename in file_mapping.values():
    filepath = os.path.join(save_directory, filename)
    if not os.path.exists(filepath):
        pd.DataFrame(columns=headers).to_csv(filepath, index=False)

# Connect and login to IMAP server
#NOTE: The IMAP4_SSL class uses SSL to encrypt the communication between the email client (your script) and the IMAP server. 
# This ensures that the data transferred <including login credentials and email content> remains secure.

mail = imaplib.IMAP4_SSL(imap_host, imap_port)
mail.login(username, password)
mail.select('inbox')

# look for unread emails with csv
typ, messages = mail.search(None, '(UNSEEN)') #NONE = charset, UNSEEN = unread emails only
if typ == 'OK': #typ is status of search operation. OK or NO/BAD
    for num in messages[0].split():
        typ, data = mail.fetch(num, '(RFC822)')
        #NOTE RFC822 Fetching with this criterion means we retrieve the entire message, including all headers, body, and attachments in their original format.
        if typ != 'OK':
            print("Failed to fetch email:", num)
            continue

            
        # Parse the email content
        # converts the raw email data retrieved by mail.fetch() into a Python email object 
        msg = email.message_from_bytes(data[0][1], policy=policy.default)
        subject = msg['Subject']
        print("Checking email:", subject)
        

        # Determine the target file based on the subject line and key phrase
        target_file = None
        for prefix, filename in file_mapping.items():
            if subject.startswith(prefix) and key_phrase in subject:
                target_file = os.path.join(save_directory, filename)
                break        
        if target_file is None:
            print("No matching file or key phrase incorrect for subject:", subject)
            continue


        # Process each part of the email
        for part in msg.walk():
            # skip the current iteration of the loop if the part is of type multipart. 
            # This is because multipart parts do not contain direct data to process but are merely structural, containing other parts.
            if part.get_content_maintype() == 'multipart':
                continue
                
            #skips the remainder of the loop for the current part if it doesn't have a Content-Disposition.
            if part.get('Content-Disposition') is None:
                continue

            # Check if the part has a filename and is a CSV file
            filename = part.get_filename()
            if filename and filename.lower().endswith('.csv'):
                filepath = os.path.join(save_directory, filename)
                with open(filepath, 'wb') as f:
                #Opens the file in write-binary ('wb') mode. Binary mode is used here to ensure that the data is written exactly as it is received.
                    f.write(part.get_payload(decode=True))
                    
                # Aggregate CSV data
                try:
                    new_data = pd.read_csv(filepath)
                    existing_data = pd.read_csv(target_file)
                    combined_data = pd.concat([existing_data, new_data])
                    combined_data.to_csv(target_file, index=False)
                    print("Aggregated data to:", target_file)
                finally:
                    os.remove(filepath)  # Remove the temporary file

mail.close()
mail.logout()


Checking email: 2 testpassword123
Aggregated data to: C:\Users\Ahern\Downloads\Python\_FINALPROJECT\2.csv


('BYE', [b'Microsoft Exchange Server IMAP4 server signing off.'])